# Проверка работы нейросети для областей одинаковых товаров
рисование картинок   
вычисление symmetric_best_dice   

In [ ]:
%load_ext autoreload
%aimport -sys, -os, -torch, -PIL, -PIL.ImageDraw, -PIL.ImageFont, -numpy as np
%autoreload 2

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Ïðîâåðêà ðàáîòû íåéðîñåòè äëÿ îáëàñòåé îäèíàêîâûõ òîâàðîâ
# ðèñîâàíèå êàðòèíîê
# âû÷èñëåíèå symmetric_best_dice

# In[1]:


from ovotools.params import AttrDict
import sys
sys.path.append('../..')
import local_config
from os.path import join
model_name = 'NN_results/retina_chars_72c04f'
model_fn = join(local_config.data_path, model_name)

params = AttrDict.load(model_fn + '.param.txt', verbose = True)
model_fn += '/models/clr.017'
params.data.net_hw = (1024,1024,) #(512,768) ###### (1024,1536) #
params.data.batch_size = 1 #######

params.model_params.encoder_params.iuo_fit_thr = 0 # if iou > iuo_fit_thr => rect fits anchor
params.model_params.encoder_params.iuo_nofit_thr = 0

params.augmentation = AttrDict(
    img_width_range=(1024, 1024),
    stretch_limit = 0.0,
    rotate_limit=0,
)

import torch
import ignite

device = 'cuda:0'


# In[3]:

import DSBI_invest.data
import create_model_retinanet
import PIL
import PIL.ImageDraw
import PIL.ImageFont
import numpy as np

def TensorToPilImage(tensor, params):
    std = (0.2,) #params.data.std
    mean = (0.5,) #params.data.mean
    vx_np = tensor.cpu().numpy().copy()
    vx_np *= np.asarray(std)[:, np.newaxis, np.newaxis]
    vx_np += np.asarray(mean)[:, np.newaxis, np.newaxis]
    vx_np = vx_np.transpose(1,2,0)*255
    return PIL.Image.fromarray(vx_np.astype(np.uint8))

model, collate_fn, loss = create_model_retinanet.create_model_retinanet(params, phase='train', device=device)
model = model.to(device)
model.load_state_dict(torch.load(model_fn))
model.eval()
print("Model loaded")

train_loader, (val_loader1, val_loader2) = DSBI_invest.data.create_dataloaders(params, collate_fn,
                    data_dir=r'D:\Programming\Braille\Data\My', fn_suffix = '', mode='inference',
                    verbose = 2)
train_loader_it = None
del train_loader_it

In [ ]:
try:
    batch = next(train_loader_it)
except (StopIteration, NameError):
    print('restarting epoch')
    train_loader_it = iter(train_loader)
    batch = next(train_loader_it)
    
data, target = ignite.engine._prepare_batch(batch, device=device)

with torch.no_grad():
    (loc_preds, cls_preds) = model(data.to(device))
loss_val = loss((loc_preds, cls_preds), target)
loss_val, loss.get_dict()

img = TensorToPilImage(data[0], params)
w,h = img.size

cls_thresh = 0.3
nms_thresh = 0

encoder = loss.encoder
boxes, labels, scores = encoder.decode(loc_preds[0].cpu().data, cls_preds[0].cpu().data, (w,h),
                              cls_thresh = cls_thresh, nms_thresh = nms_thresh)

draw = PIL.ImageDraw.Draw(img)
fnt = PIL.ImageFont.truetype("arial.ttf", 8)
fntA = PIL.ImageFont.truetype("arial.ttf", 12)
for i, box in enumerate(boxes):
    draw.rectangle(list(box), outline='green')
    int_lbl = labels[i].item()
    lbl = DSBI_invest.data.int_to_letter(int_lbl, 'RU')
    #lbl = '' #GVNC
    if not lbl:
        lbl = DSBI_invest.data.int_to_label(int_lbl)
        lbl = lbl[:3]+'.'+lbl[3:]
    score = scores[i].item()
    score = '{:.1f}'.format(score*10)
    draw.text((box[0],box[3]), lbl, font=fntA, fill="white")
    draw.text((box[0],box[3]+12), score, font=fnt, fill='green')
img

In [ ]:
cls_thresh = 0.3
nms_thresh = 0

encoder = loss.encoder
boxes, labels = encoder.decode(loc_preds[0].cpu().data, cls_preds[0].cpu().data, (w,h),
                              cls_thresh = cls_thresh, nms_thresh = nms_thresh)

print
img = TensorToPilImage(data[0], params)
draw = PIL.ImageDraw.Draw(img)
for box in boxes:
    draw.rectangle(list(box), outline='red')
img



In [ ]:
loc_preds[0].cpu().data.shape, cls_preds[0].shape, \
loc_preds[0].cpu().data.min(), cls_preds[0].min(), \
loc_preds[0].cpu().data.max(), cls_preds[0].max()

In [ ]:
target[1][0].unsqueeze(1).shape

In [ ]:
target[1][0].shape

In [ ]:
encoder.iuo_fit_thr

In [ ]:
boxest, labelst, _ = encoder.decode(target[0][0].cpu().data, target[1][0].cpu().clamp(0,1).unsqueeze(1).float().data, (w,h),
                              cls_thresh = 0, nms_thresh = 0)

draw = PIL.ImageDraw.Draw(img)
for box in boxest:
    draw.rectangle(list(box), outline='green')
img